## Домашнее задание №4. Классификатор отзывов к фильмам
В рамках данного домашнего задания мы разработаем систему классификации отзывов к фильмам на положительные и отрицательные. Такая система может быть полезна, например, компаниям, которые анализируют отзывы в социальных сетях и решают, стоит ли снимать продолжение к фильму.

Данная задача по сути - задача на работу с естественным языком. Несмотря на то, что техники, которые мы будем применять, довольно простые, они позволят проиллюстрировать как задачу классификации, так и несколько основных идей из области анализа текстов. 

### Feature engineering
Как мы уже успели рассмотреть на занятиях, методы машинного обучения обычно работают с признаковым представлением объекта. Выделить признаки, на которых можно обучать модели, не всегда очень легко. Задачу выделения признаков неформально называют "feature engineering", и выделение хороших признаков зачастую оказывается более сложной задачей, чем собственно построение модели машинного обучения

В рамках данного домашнего задания мы покажем, как можно выделить признаки из текстовой информации, чтобы построить классификатор текстов. Более подробно мы рассмотрим тему feature engineering в следующем модуле. 

### Модель "Мешок Слов"

Одной из простейших идей при работе с текстами является так называемая модель "мешка слов" (bag of the words, bow).
Идея заключается в том, что порядок слов для тематики не важен, а важны только сами слова. 

Частота встречающихся слов в этом подходе будет теми самыми признаками, которые мы можем использовать для обучения классификатора. 

Это, конечно, не совсем правда; например, фразы "мне не понравился фильм" и "не мне понравился фильм"
отличаются только порядком слов, но имеют совершенно разный смысл. 

Тем не менее, как показывает практика,  такого подхода достаточно, чтобы определить тональность или тематику текста.



Определим функцию преобразования текста в мешок слов. Слово будем определять, как последовательность русских букв, и будем искать их при помощи регулярных выражений (про регулярные выражения можно почитать тут: https://habr.com/post/349860/)

Весь текст будем приводить к нижнему регистру, чтобы слова, написанные в разных регистрах, не отличались друг от друга. 

In [1]:
import re

def text2bow(text):
    result = dict()
    word_regexp = re.compile(r'[а-я]+')
    for word in word_regexp.findall(text.lower()):
        if word not in result:
            result[word] = 1
        else:
            result[word] = result[word] + 1
    return result
            
print(text2bow('Мне не понравился фильм, а ему понравился фильм'))

{'мне': 1, 'не': 1, 'понравился': 2, 'фильм': 2, 'а': 1, 'ему': 1}


Мы возвращаем словарь с частотами слов. Для того, чтобы работать с моделями машинного обучения, нам нужно "табличное" представление даннных. Это очень просто сделать при помощи pandas, так как в нем уже реализовано преобразование словарей в DataFrame:

In [2]:
import pandas as pd
docs = [
            text2bow('Мне не понравился фильм, а ему понравился фильм'), 
            text2bow('Фильм был очень хороший! Мне понравился'), 
       ]
df = pd.DataFrame(docs)
df

,а,был,ему,мне,не,очень,понравился,фильм,хороший
0,1.0,NaN,1.0,1,1.0,NaN,2,2,NaN
1,NaN,1.0,NaN,1,NaN,1.0,1,1,1.0


Пустоты лучше заполнить нулями:

In [3]:
df = df.fillna(0.0)
df

,а,был,ему,мне,не,очень,понравился,фильм,хороший
0,1.0,0.0,1.0,1,1.0,0.0,2,2,0.0
1,0.0,1.0,0.0,1,0.0,1.0,1,1,1.0


Мы преобразовали тексты к "табличному" виду!
Теперь к ним можно применять модели машинного обучения.

Теперь давайте загрузим настоящие данные. Работать будем с настоящими отзывами к фильмам. Мы заранее привели их к модели "мешок слов". Всего в вашей выборке 30000 примеров. 

Мы отфильтровали 10000 самых популярных слов на русском языке, чтобы данных не получилось слишком много.

Колонка is_positive содержит целевую функцию (положительный отзыв или отрицательный).

Внимание! Данных довольно много, их загрузка займет какое-то время.

In [4]:
data = pd.read_csv('train.csv', dtype='uint8')
data.head()

,is_positive,а,абсолютная,абсолютно,абсолютное,абсолютной,абсурд,абсурда,аварии,австралии,...,ярко,яркого,яркое,яркой,яркую,ярости,ярость,ярче,ясно,ящик
0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,1,3,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [5]:
labels = data['is_positive']
del(data['is_positive'])

# Задание №1 

Разделите выборку на тестовую и контрольную при помощи функции train_test split.
Не забудте импортировать функцию train_test_split из модуля sklearn.metrics.
Пример можно посмотреть в предыдущем домашнем задании или в видео про разделение выборки на train и test

In [6]:
#подставьте нужные вызовы вместо слов в угловых скобках
from sklearn.model_selection import train_test_split

train_data, test_data, train_labels, test_labels = train_test_split(data, labels)

# Задание №2

##### Обязательная часть
Обучите модель классификации. 
Вам необходимо импортировать модель машинного обучения из библиотеки sklear, обучить ее и проверить на тестовых данных. Вы можете попробовать разные модели. 

Мы рекомендуем попробовать следущие модели:

- DecisionTreeClassifier из модуля sklearn.tree
- RandomForestClassifier из модуля sklearn.ensemble

Для получения зачета нужно получить значение метрики AUC не менее 0.7
##### Необязательная часть 1
Для получения зачета вам достаточно просто построить одну из моделей с параметрами по умолчанию, но, чтобы улучшить свой результат, вы можете попробовать задать параметры моделей. У модели RandomForestClassfier для улучшения качества можно задавать, например, такие параметры, как: 
- Количество деревьев(n_estimators)
- Максимальную высоту дерева(max_depth)

Полный список критериев можно посмотреть в оффициальной документации:
http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

##### Необязательная часть 2
Вы также можете попробовать самостоятельно найти другие модели классификации для достижения еще лучшего качества модели. Можете поискать, например, как построить Логистическую Регрессию или Градиентный Бустинг.

In [11]:
# подставьте нужные значения модуля и модели
from sklearn.ensemble import GradientBoostingClassifier

# подставьте создайте модель
model = GradientBoostingClassifier(n_estimators=200, random_state = 0)
model.fit(train_data, train_labels)
prediction = model.predict_proba(test_data)[:,1]

from sklearn.metrics import roc_auc_score
auc = roc_auc_score(test_labels, prediction)
print("auc:", auc)

auc: 0.9002058232533453


### Оценивание задания.

Выполните предсказания для контрольных данных. Скопируйте и вставьте результат в систему в EdX на странице "Домашнее задание".

Критерии оценки:
    - AUC меньше 0.7 - незачет
    - AUC от 0.7 до 0.8 - 0.8 балла
    - AUC от 0.8 до 0.9 - 0.9 балла
    - AUC больше 0.9 - 0.9 

In [12]:
data_control = pd.read_csv('control.csv')
prediction = model.predict_proba(data_control)[:,1]
print(','.join("{:.3f}".format(p) for p in prediction))

0.935,0.773,0.896,0.972,0.968,0.934,0.954,0.548,0.916,0.647,0.169,0.929,0.755,0.983,0.903,0.955,0.897,0.941,0.668,0.947,0.909,0.957,0.939,0.961,0.836,0.930,0.746,0.787,0.780,0.912,0.974,0.986,0.812,0.962,0.881,0.946,0.922,0.969,0.800,0.933,0.926,0.936,0.950,0.881,0.890,0.853,0.880,0.401,0.950,0.821,0.788,0.890,0.961,0.911,0.928,0.961,0.569,0.945,0.873,0.919,0.944,0.800,0.900,0.956,0.980,0.954,0.946,0.900,0.956,0.920,0.923,0.948,0.956,0.922,0.896,0.960,0.934,0.956,0.851,0.871,0.937,0.536,0.967,0.951,0.923,0.940,0.946,0.828,0.940,0.918,0.975,0.960,0.961,0.979,0.968,0.959,0.950,0.896,0.908,0.952,0.977,0.937,0.956,0.878,0.974,0.957,0.856,0.710,0.815,0.932,0.980,0.907,0.964,0.916,0.932,0.916,0.881,0.128,0.941,0.936,0.947,0.913,0.934,0.949,0.476,0.367,0.825,0.972,0.870,0.890,0.972,0.934,0.979,0.383,0.508,0.903,0.977,0.932,0.823,0.866,0.941,0.557,0.950,0.956,0.941,0.946,0.924,0.885,0.883,0.924,0.964,0.851,0.959,0.962,0.962,0.865,0.880,0.872,0.957,0.931,0.378,0.736,0.959,0.912,0.919,0.942,0.52

Выполните строчку выше, скопируйте и вставьте результат в систему в EdX.

# Дополнительная информация

Наверное, вам интересно посмотреть, как ваша модель классифицирует реальные отзывы?

Ниже приведен код, при помощи которого это можно сделать. Мы преобразуем отзыв в мешок слов и подаем на вход модели. 
Отличие от того, что мы делали ранее, - в модели зафиксирован порядок признаков, поэтому мы передаем дополнительный параметр columns.

In [165]:
review = "Лучший фильм, который я когда-либо видел!"


bows = [text2bow(review)]
data_my = pd.DataFrame(bows, columns=train_data.columns).fillna(0.0)
score = model.predict_proba(data_my)[:,1][0]
print("Результат автоматической классификации: {:.3f}".format(score))

Результат автоматической классификации: 0.965


Модель мешка слов, которую мы с вами использовали, является довольно простой моделью. Более сложные модели используют порядок слов, а также приводят слова к нормальному виду.

Наиболее продвинутые современные методы базируются на рекуррентных и сверточных нейросетях. См. например: https://habr.com/company/dca/blog/274027/